# AdaBoost





## Learning objective

After reading this notebook, students will be able:

- Explain the intuition behind AdaBoost for binary classification.
- Understand how the models are combined in AdaBoost.
- Explain how weights are assigned for each data points in AdaBoost.
- Understand the reason behind using an exponential loss function.


In the previous reading material, we studied how boosting works—the concept of additively combining many weak learners. Recall each sequentially added learner's goal is to compensate for the error/shortcoming of an existing model. We also discussed a general boosting algorithm. As discussed, there are two popular boosting algorithms: __AdaBoost__ and Gradient Boosting. We will cover Gradient Boosting in the next chapter. In this chapter, we will first understand the intuition behind Adaptive Boosting( AdaBoost ) with some visualization. Later, we will perform the mathematical formulation of the AdaBoost algorithm.




## Introduction

AdaBoost is one of the first and most popular boosting algorithms. Initially, it was formulated for a binary classification task. Later, it was modified and used for the regression task too. In this chapter, we will discuss AdaBoost for binary classification.

Recall that the iterative addition of the successive model is based on an existing model's error or shortcoming. Also, recall that the dataset is modified based on the error or shortcoming of an existing model. The data modification process in AdaBoost consists of assigning the weights $w_1, w_2, \ldots, w_N$ to each of the training instances $\big(\mathbf{x}_i, y_i\big)_{i=1}^{N}$ and modifying them based on the performance of each successively added model. 

Initially, all the training instances are assigned equal weights, $w_i = \frac{1}{N}$. The first model (weak learner) is trained on an unmodified dataset (in which all instances have equal weight and are treated equally). But for successive iterations, each instance's weight is modified based on the performance of a previously added model. The weights are increased for those instances that were misclassified by the previously added model, and decreased for correctly classified instances. 

Further models are trained considering the new (modified) weights of instances. This means successive models should focus or concentrate on the misclassified instances with higher weights.

## Intuition

In the above section, we discussed the general intuition behind AdaBoost. Let us make this intuition more apparent with the help of visualization. Here we will understand the working of AdaBoost with the help of a simple binary classification problem.

__Note:__ The images used for intuition are inspired from [A Boosting Tutorial](https://www.csie.ntu.edu.tw/~mhyang/course/u0030/papers/schapire.pdf) by Rob Schapire.

The figure below shows the scatter plot of the training dataset(dataset is binary).
The label of each instance is either positive or negative class as represented by "+", or "-" symbol respectively. The size of symbol represents the weight of instance. Initially, each instance has equal weight, so equal size. The original dataset is labelled as $D_1$.
<div align="center">
<figure>
<img src="https://i.postimg.cc/XJbnVZVD/image.png" width="20%" alt="Original binary classification dataset">
<figcaption>Figure 1. Original binary classification dataset</figcaption>
</figure>
</div>



We train a weak learner $h_1$, say a decision tree stump (which returns 1 for positive class and -1 for negative class) on the original dataset $D_1$. The addition of the first weak learner forms the first iteration. Suppose the learner $h_1$ produces a decision surface, as shown in the figure. Here, three positive instances are misclassified. These instances are encircled. The weight of these three instances increases while other instances' weight decreases as indicated by the modified dataset $D_2$ in the figure below. The size of symbols indicates the weight of instances. Notice that in $D_2$, the size of three misclassified positive instances increases. Here, $\alpha_1 = 0.3$ represents the contribution factor of a learner $h_1$. The accuracy of the learner determines the value of $\alpha$. We will see how to compute $\alpha$ later.
<div align="center">
<figure>
<img src="https://i.postimg.cc/zG7PM99H/image.png" width = "40%" >
<figcaption>Figure 2:  Figure showing the decision boundary in  original dataset $D_1$ modified dataset $D_2$ in first iteration.
</figcaption>
</figure>
</div>
<div align="center">

</div>






We again train a new decision stump $h_2$ on modified dataset $D_2$ in the second iteration. The goal of this model is to concentrate on the error of model $h_1$. Suppose $h_2$ makes the decision boundary as shown in the figure.
<div align="center">
<figure>
<img src="https://i.postimg.cc/28KVkJQg/image.png" width="60%" alt="Decision boundary in D1 and D2, and modified D3">
<figcaption>Figure 3: Figure showing the decision boundary in original dataset $D_1$ and modified dataset $D_2$. The figure also shows dataset $D_3$ modified in second iteration .
</figcaption>
</figure>
</div>


 Observe that the model $h_2$ has correctly classified the instances misclassified by $h_1$. As seen in the above figure, three negative instances, highlighted by a circle, are misclassified. The weight of these instances should be increased, and the weight of the rest should be decreased. The dataset $D_3$ shows the dataset modified after an update of the weight of instances.


Now, we again train a decision stump $h_3$ on dataset $D_3$ as indicated by decision boundary in figure. Again, the misclassified instances are highlighted by a circle.
<div align="center">
<figure>
<!-- <img src="https://doc.google.com/a/fusemachines.com/uc?export=download&id=1lUuIeLsknbQiQt7p2LzK-G788cGMCmUB" width = "60%"> -->
<img src="https://i.postimg.cc/28KVkJQg/image.png" width = "60%">
<figcaption>Figure 4: Figure showing the decision boundary in original $D_1$ and modified dataset $D_2$ and $D_3$.
</figcaption>
</figure>
</div>


 Based on the error of model $h_3$, we can update the weight of instances and add another model. For now, let's stop the addition of further models. Let us see how to combine these models $h_1, h_2$, and  $h_3$. While combining these models, each model is weighted by the contribution factor. The contribution factor determines how good each model is while making a prediction. The combination of three models is shown as:

<div align="center">
<figure>
<!-- <img src="https://doc.google.com/a/fusemachines.com/uc?export=download&id=1SCyPqnsascIKcfszP7dKBlC8_iCoRHNc" > -->
<img src="https://i.postimg.cc/Kzyq3025/image.png" width="60%" alt="Ensembled Classifier">
<figcaption>Figure 5. Ensembled Classifier</figcaption>
</figcaption>
</figure>
</div>

Notice how decision surface are combined in the resultant boosted model $F$. This indicate that a complex function(model) can be obtained by the combination of simpler models.


Mathematically, the addition of these models can be expressed as:

$$F(\mathbf{x}) = \text{sign}\big(0.42 \cdot h_1(\mathbf{x}) + 0.65 \cdot h_2(\mathbf{x}) + 0.92 \cdot h_3(\mathbf{x})\big)  \tag{i}$$


Here, __sign()__ is a step function. The step function is defined as:

\begin{equation}
  \text{sign}(x) =
    \begin{cases}
      1 & \text{if $x\ge 0$}\\
       -1 & \text{if $x \lt 0$}
    \end{cases}       
\end{equation}

__Note :__ $h_t({\mathbf{x}})$ gives either 1 or -1 as a prediction for instance $\mathbf{x}$.

The above equation shows that the ensemble considers much of the contribution (0.92>0.42 & 0.65) from the model $h_3$ compared to the other models while making a prediction.





### Inference in AdaBoost
Until now, we saw the working of the AdaBoost algorithm. In this section, we will see how to make inference/prediction for any given instance.
Let us predict the class/label of an instance highlighted by red circle in the figure below.
<div align="center">
<figure>
<!-- <img src="https://doc.google.com/a/fusemachines.com/uc?export=download&id=1d_SFG6jK8CZDOhveQKqdqjrNPREJyl4u" width = "20%" > -->
<img src="https://i.postimg.cc/ZqQYThPQ/image.png" width ="20%">
<figcaption>Figure 6: Inference of an instance highlighted by red circle.
</figcaption>
</figure>
</div>


 Let us represent this instance as $\mathbf{x}_1$. The prediction made by learners(decision stumps) $h_1, h_2$, and $h_3$ for the instance $\mathbf{x}_1$ as seen from the  decision boundary in figure 5 are:
$$h_1(\mathbf{x}_1) = 1$$
$$h_2(\mathbf{x}_1) = 1$$
$$h_3(\mathbf{x}_1) = -1$$

The prediction of an ensemble model $F$ is:

  <!-- > $F(\mathbf{x}_1) = \text{sign}\bigg{(}0.42*h_1(\mathbf{x}_1)+0.65*h_2(\mathbf{x}_1)+0.92*h_3(\mathbf{x}_1)\bigg{)}  $

 > $F(\mathbf{x}_1) = \text{sign}(0.42*1+0.65*1+ 0.92*-1)$

 > $F(\mathbf{x}_1) = \text{sign}(+0.15) = 1 \quad \quad \because \text{sign() is a step function}$ -->

\begin{align*}
F(\mathbf{x}_1) &=  \text{sign}\big(0.42 \cdot h_1(\mathbf{x}_1) + 0.65 \cdot h_2(\mathbf{x}_1) + 0.92 \cdot h_3(\mathbf{x}_1)\big) \\
F(\mathbf{x}_1) &=  \text{sign}(0.42 \times 1 + 0.65 \times 1 + 0.92 \times (-1)) \\
F(\mathbf{x}_1) &=  \text{sign}(0.15) = 1 \qquad \text{(since sign() is a step function)}
\end{align*}


In this way, we understood the intuition behind the AdaBoost algorithm. In the next section, we will perform the mathematical formulation of the AdaBoost algorithm.

## Adaptive Boosting(AdaBoost)

As discussed, AdaBoost is one of the oldest boosting algorithms, and it was initially formulated as a simpler algorithm to solve a binary classification problem.
 Later, AdaBoost was reformulated as a stagewise forward additive modeling using an exponential loss function. We will discuss the reason behind using an exponential loss function later. We have already discussed forward stagewise additive modeling in the previous chapter.

The exponential loss function is given as:
$$L(y, F(\mathbf{x})) = \text{exp}(-yF(\mathbf{x})) \tag{1}$$

Using forward stagewise additive modeling, we must solve

$$\alpha_t, h_t = \arg \underset{\alpha, h}{\min}  \sum_{i= 1}^N L\big(y_i, F_{t-1}(\mathbf{x}_i)+ \alpha h(\mathbf{x}_i)\big) \tag{2}$$

to find the model $h_t$ and it's corresponding $\alpha_t$ to be added at each iteration.



For exponential loss function, we can rewrite equation $(2)$ as:

$$
\alpha_t, h_t = \arg \underset{\alpha, h}{\min}  \sum_{i= 1}^N \exp\left[-y_i\left( F_{t-1}(\mathbf{x}_i)+ \alpha h(\mathbf{x}_i)\right)\right]  \tag{3}
$$

$$
\alpha_t, h_t = \arg \underset{\alpha, h}{\min}  \sum_{i= 1}^N \exp\left(-y_i F_{t-1}(\mathbf{x}_i)\right) \cdot \exp\left(-\alpha y_i h(\mathbf{x}_i)\right)
$$

The equation $(3)$ can be further expressed as:

$$
\alpha_t, h_t = \arg \underset{\alpha, h}{\min}  \sum_{i= 1}^N w_i^{(t)} \exp\left(-\alpha y_i h(\mathbf{x}_i)\right)  \tag{4}
$$

where,

$$
w_i^{(t)} = \exp\left(-y_i F_{t-1}(\mathbf{x}_i)\right)
$$

is dependent on each instance and is regarded as a weight. Also note that the weight also depends on the existing model $F_{t-1}(\mathbf{x})$, so the weight of individual training instances changes in each iteration $t$.

It is difficult to obtain $\alpha_t$ and $h_t$ simultaneously. We solve equation $(4)$ in two steps: first we obtain the optimal $h_t$, and use it to obtain the corresponding $\alpha_t$.

Please carefully observe the expression $-\alpha y_ih(\mathbf{x}_i)$ inside the exponential function in equation $(4)$.

Here, $\alpha \gt0$ , $y_i\in \{-1, 1\}$, and $h(\mathbf{x}_i) \in \{-1, 1\}$.

So, the expression $-\alpha y_ih(\mathbf{x}_i)$ can be expressed as:

\begin{equation}
  -\alpha y_ih(\mathbf{x}_i) = \tag{5}
    \begin{cases}
      -\alpha & \text{if $y_i = h(\mathbf{x}_i)$}\\
       \alpha & \text{if $y_i \neq h(\mathbf{x}_i)$}
    \end{cases}       
\end{equation}







Using above expression, the minimization term in equation $(4)$ can be rewritten as:

$$e^{-\alpha}. \sum_{y_i=h(\mathbf{x}_i)} w_i^{(t)} + e^{\alpha}.\sum_{y_i \neq h(\mathbf{x}_i)} w_i^{(t)} \tag{6}$$

Note the term $\sum_{y_i=h(\mathbf{x}_i)} w_i^{(t)} $ and $\sum_{y_i\neq h(\mathbf{x}_i)} w_i^{(t)}$ are reciprocal to each other, i.e.,

$$ \sum_{y_i=h(\mathbf{x}_i)} w_i^{(t)} = 1- \sum_{y_i\neq h(\mathbf{x}_i)} w_i^{(t)}$$

Putting this expression in equation $(6)$, we have,

$$(e^{\alpha} - e^{-\alpha}) \sum_{i=1}^{N} w_i^{(t)} I\big(y_i \neq h(\mathbf{x}_i)\big) + e^{-\alpha} \sum_{i=1}^N w_i^{(t)}$$

where, $I$ is an indicator function such that
\begin{equation}
  I(y_i \neq h(\mathbf{x}_i)) =
    \begin{cases}
      1 & \text{if $y_i \neq h(\mathbf{x}_i)$}\\
       0 & \text{if $y_i = h(\mathbf{x}_i)$}
    \end{cases}       
\end{equation}

Here, $\sum_{i=1}^N w_i^{(t)} = 1$ , and for any given value of $\alpha$ the expressions $(e^{\alpha} - e^{-\alpha})$ and $e^{-\alpha}$ are constant for a given iteration.

The minimization expression in equation $(4)$ can be re-expressed as:

$$ h_t = \arg \underset{h}{\min}  \sum_{i= 1}^N w_i^{(t)} I(y_i \neq h(\mathbf{x}_i)) \tag{7}$$

This equation represents a classifier that minimizes the weighted error rate.

__Note :__ The error rate of a classifier $h$ trying to predict $y$ is given as:
$$\text{err} = \frac{1}{N} \sum_{i=1}^N I(y_i\neq h(\mathbf{x}_i))$$

We can put the value of $h_t$ in equation $(4)$ we can solve for the value of $\alpha_t$. The value of $\alpha_t$ obtained is:

$$\alpha_t = \frac{1}{2} log\frac{1-\text{err}_t}{\text{err}_t}$$

where,

$\text{err}_t$ is a weighted error rate of a classifier, which we have minimized in equation $(7)$.

### Model Update

Once we obtain the classifier $h_t$ and its corresponding contribution factor $\alpha_t$, we can update our model as:

$$F_t(\mathbf{x}) = F_{t-1}(\mathbf{x}) + \alpha_t h_t(\mathbf{x})$$

### Weight Update

We need to update the weight of instances in each iteration. Note the weight corresponds to the difficulty in classifying the instances correctly. Higher is the difficulty in classifying the instance correctly; higher is the weight.

 The weight for the next iteration is computed based on the performance of a currently added model. The weight update of each instance for the next iteration $t+1$ is performed as:
$$
w_i^{(t+1)} = w_i^{(t)} \exp\big(-\alpha_t y_i h_t(\mathbf{x}_i)\big)
$$

As seen from equation $(5)$, the value of expression $-\alpha_t y_i h_t(\mathbf{x}_i)$ is negative for correct prediction and positive for incorrect prediction. This clearly shows that correctly predicted instances increases while the weight of incorrectly predicted instances decreases.


### Why exponential loss?

Until now, we understood the intuition behind the AdaBoost, and mathematically formulated the AdaBoost algorithm. For optimization, we used an exponential loss function. Let us see the reason behind using an exponential loss function for the optimization purpose.

 In AdaBoost, we need to assign a high weight to the misclassified instances and low to the correctly classified instances.  The exponential loss function indicated by equation $(1)$ is shown in the figure below.

 <div align="center">
<figure>
<img src="https://i.postimg.cc/bJ313r15/image.png" width = "50%">

<figcaption>Figure 7: Exponential loss function
</figcaption>
</figure>
</div>

The above plot shows $yF(\mathbf{x})$ on X-axis and corresponding exponential loss on Y-axis. For correct predictions, $yF(\mathbf{x}) \gt 0$ for correct prediction and for incorrect predictions  $yF(\mathbf{x}) \lt 0$. The above curve shows that the loss is high for incorrect predictions and low for correct predictions. These losses are incorporated in the dataset by updating the weights.






## AdaBoost for Regression

Until now, we have discussed AdaBoost for binary classification. We used an exponential loss function for binary classification. The concept of AdaBoost can be used to solve a regression problem too. For the regression problem, we should use other loss functions like a squared loss. We have already discussed squared loss in the linear regression module. Apart from the squared loss function, each base learner $h_t$ should return a real-valued output rather than a binary value $\{-1, 1\}$ as in binary classification. The rest of an algorithm for regression remains the same as for classification.


In this way, we are at the end of this chapter. We discussed the AdaBoost algorithm for binary classification with simple visualization and mathematically formulated an algorithm. We also discussed a gentle overview of AdaBoost for regression. In the next chapter, we will discuss another popular technique of boosting called Gradient Boosting.







## Key Takeaways

- The errors/shortcomings of an existing model is identified by high weights instances.

- Each successively added learner should concentrate on the instances misclassified by the previous learner.

- The weight of instances are updated in each iteration.

- Exponential loss function is used to model the shortcomings of a previous model in binary classification; for regression, we can use squared loss function.





